# Target Wait Time, Minimization    

In [44]:
from gurobipy import Model, GRB, max_, LinExpr
import pandas as pd


#Read In Surgical Information
surgicalPortfolio = pd.read_csv('surgeries_data.csv')
surgeries = surgicalPortfolio['id'].unique().tolist()

#read in capacity information
caps = pd.read_csv('OR_caps.csv')
rooms = caps['room'].unique().tolist()
days = caps['day'].unique().tolist()

m = Model('WaitTimeMinimization')

# Create Binary Variable for if surgery is scheduled in room on day 
surgery_scheduled = m.addVars(surgeries, days, rooms, vtype=GRB.BINARY, name="schedule")

# Add excess wait time variables
excess_wait_times = m.addVars(surgeries, vtype=GRB.INTEGER, name="excess_wait_time")

# Calculate actual start days based on the scheduling decision
actual_start = m.addVars(surgeries, vtype=GRB.INTEGER, name="actual_start")

#constraints:
#surgeries can only be scheduled once -> sum of all variables for each surgery id <=1
for s in surgeries:
    m.addConstr(sum(surgery_scheduled[s, d, r] for d in days for r in rooms) <= 1, name=f"OneTime_{s}")

#capacity constraint -> the sum of the durations of surgeries scheduled in a room on a given day cannot exceed the room's capacity
for r in rooms:
    for d in days:
        # Calculate total hours scheduled in room 'r' on day 'd'
        total_scheduled_hours = sum(surgery_scheduled[s, d, r] * surgicalPortfolio.loc[surgicalPortfolio['id'] == s, 'duration'].values[0]
                                    for s in surgeries)
        
        # Get the capacity of room 'r' on day 'd'
        room_capacity = caps.loc[(caps['room'] == r) & (caps['day'] == d), 'capacity'].values[0]
        
        # Add a constraint that the total scheduled hours must not exceed the room capacity
        m.addConstr(total_scheduled_hours <= room_capacity, name=f"CapacityLimit_{r}_{d}")


# Constraint to calculate actual start days (assuming 'days' is a sorted list)
for s in surgeries:
    m.addConstr(actual_start[s] == sum(d * surgery_scheduled[s, d, r] for d in days for r in rooms),
                name=f"CalcActualStart_{s}")


# Objective function: Minimize the sum of all excess wait times
objective = LinExpr()
for s in surgeries:
    objective += max_(0, excess_wait_times[s])


m.setObjective(objective, GRB.MINIMIZE)

# Output the results
if m.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    for s in surgeries:
        for d in days:
            for r in rooms:
                if surgery_scheduled[s, d, r].X > 0.5:  # Check if the surgery is scheduled
                    print(f"Surgery {s} is scheduled in room {r} on day {d}.")
else:
    print("No feasible solution found.")




TypeError: unsupported operand type(s) for +=: 'gurobipy.LinExpr' and 'GenExprMax'

In [38]:
for s in surgery_scheduled:
    print(s)

('S1', 1, 0)
('S1', 1, 1)
('S1', 1, 2)
('S1', 2, 0)
('S1', 2, 1)
('S1', 2, 2)
('S1', 3, 0)
('S1', 3, 1)
('S1', 3, 2)
('S1', 4, 0)
('S1', 4, 1)
('S1', 4, 2)
('S1', 5, 0)
('S1', 5, 1)
('S1', 5, 2)
('S1', 6, 0)
('S1', 6, 1)
('S1', 6, 2)
('S1', 7, 0)
('S1', 7, 1)
('S1', 7, 2)
('S1', 8, 0)
('S1', 8, 1)
('S1', 8, 2)
('S1', 9, 0)
('S1', 9, 1)
('S1', 9, 2)
('S1', 10, 0)
('S1', 10, 1)
('S1', 10, 2)
('S1', 11, 0)
('S1', 11, 1)
('S1', 11, 2)
('S1', 12, 0)
('S1', 12, 1)
('S1', 12, 2)
('S1', 13, 0)
('S1', 13, 1)
('S1', 13, 2)
('S1', 14, 0)
('S1', 14, 1)
('S1', 14, 2)
('S1', 15, 0)
('S1', 15, 1)
('S1', 15, 2)
('S1', 16, 0)
('S1', 16, 1)
('S1', 16, 2)
('S1', 17, 0)
('S1', 17, 1)
('S1', 17, 2)
('S1', 18, 0)
('S1', 18, 1)
('S1', 18, 2)
('S1', 19, 0)
('S1', 19, 1)
('S1', 19, 2)
('S1', 20, 0)
('S1', 20, 1)
('S1', 20, 2)
('S1', 21, 0)
('S1', 21, 1)
('S1', 21, 2)
('S1', 22, 0)
('S1', 22, 1)
('S1', 22, 2)
('S1', 23, 0)
('S1', 23, 1)
('S1', 23, 2)
('S1', 24, 0)
('S1', 24, 1)
('S1', 24, 2)
('S1', 25, 0)
('S1'